<a href="https://colab.research.google.com/github/NileshCT/Machine_Learning_Models/blob/main/Node_creat_for_kedro_pipeline_On_WeatherAUS_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Importing required libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import plotly.express as px

In [2]:
def get_data():
  """importing dataset
  output=importing dataset and converting into dataframe."""
  df=pd.read_csv("/content/drive/MyDrive/weatherAUS.csv.zip")
  return df

In [3]:
df=get_data()

In [5]:
def data_treatment(df):
  df.drop(["Date"],axis=1,inplace=True)
  return df
data=data_treatment(df)

In [78]:
def target_nan(df):
  a=[(i,index) for i,index in enumerate(df["RainTomorrow"]) if pd.isna(index)]
  t= pd.DataFrame(a).set_index(0)
  return t

In [87]:
t=target_nan(df)
t

,1
0,
14,NaN
283,NaN
435,NaN
437,NaN
443,NaN
...,...
144955,NaN
145091,NaN
145092,NaN


In [88]:
#Inference data extraction.
def inference(t):
  index_nan=t.index.tolist()
  df_inference=df.iloc[index_nan]
  return df_inference
df_inference=inference(result)

In [89]:
#get training data for modelling.
def training_data(df_inference):
  index_nan=t.index.tolist()
  b=[]
  for i in range(df.shape[0]):
    if i not in index_nan:
      b.append(i)
  return b


In [91]:
c=training_data(df_inference)

In [92]:
def get_training_data(b):
  df_training_data=df.iloc[b]
  return df_training_data

In [ ]:
df_training_data=get_training_data(b)
df_training_data

In [98]:
def treat_missing(df_training_data):
  df_treat_training_data=df_training_data.fillna(method="ffill",axis=0).fillna(method="bfill",axis=0)
  return df_treat_training_data
df_treat_training_data=treat_missing(df_training_data)

In [103]:
def training_data_split(df_treat_training_data):
  X_training=df_treat_training_data.drop(["RainTomorrow"],axis=1)
  y_training=df_treat_training_data["RainTomorrow"]
  return X_training, y_training
X_training, y_training=training_data_split(df_treat_training_data)

In [ ]:
from sklearn import preprocessing
L_Encoder=preprocessing.LabelEncoder()
def lebal_encoder(X_training):
  X_training["Location"]=L_Encoder.fit_transform(X_training["Location"])
  X_training["WindGustDir"]=L_Encoder.fit_transform(X_training["WindGustDir"])
  X_training["WindDir9am"]=L_Encoder.fit_transform(X_training["WindDir9am"])
  X_training["WindDir3pm"]=L_Encoder.fit_transform(X_training["WindDir3pm"])
  X_training["RainToday"]=L_Encoder.fit_transform(X_training["RainToday"])
  return X_training
X_training=lebal_encoder(X_training)


In [ ]:
X_training